-------------------------------------------------------------------------------

CHIBO 2024

An attempt to static calculate the forces in (x,y,z) at all the suspension pickup points for various roll/jounce effects

18.12.2024: Created

03.01.2025: Outboard points calc working, moved functions to susprog.py and added loop 

05.01.2025: Added jounce updater for the inner points and included in susprog.py

08.01.2025: Added trail & scrub radius calc in loop

09.01.2025: Rewrote roll centre code to correct errors in original

11.01.2025: Added in trail/scrub/KPI/Caster angle evaluation. Bug was present for a while which miscalculated RC via unsmooth rotation of inner points. Went away for unknown reason

19.01.2025: Changed units to mm, this fixes small number rounding error which causes output instability

05.03.2025: Added force calculator to add reaction loads at outer points. Refractored sim code to be a callable function to run multiple simulations

-------------------------------------------------------------------------------

In [8]:
#Import required libraries
import numpy as np
import pandas as pd
from scipy.optimize import minimize

import plotly.graph_objects as go
import matplotlib.pyplot as plt

import susprog                      # Python file containing the equations for the loop 

In [9]:
# Declare motions for evaluation

# Define maximum roll and jounce inputs
body_roll = -5.00  # Maximum roll angle in degrees
body_jounce = 0  # Maximum jounce in meters (5 cm = 0.05)
sim_vales = 100

# Generate roll and jounce motion inputs
roll_angles = np.linspace(0, np.radians(body_roll), sim_vales)  # 0 to max roll angle in 100 steps
jounce_displacements = np.linspace(0, body_jounce, sim_vales)  # 0 to max jounce in 100 steps

In [10]:
# Suspension pickup points in millimeters
upper_a_arm = np.array([
    [1.5, 506.5, 315],      # Outer point (converted from m to mm)
    [-135, 240, 220],       # Inner leading point
    [180, 255, 255]         # Inner trailing point
], dtype=np.float64)

lower_a_arm = np.array([
    [-17.5, 519, 137],      # Outer point
    [-180, 165, 87],        # Inner leading point
    [180, 170, 87]          # Inner trailing point
], dtype=np.float64)

tie_rod = np.array([
    [60, 515, 160],         # Outer point
    [70, 195, 100]          # Inner point
], dtype=np.float64)

tire_cop = np.array([-5, 525, 0])
force_vector = np.array([5000, 5000, -20000])  # Force in N


In [11]:
# Calculate lengths for upper and lower A-arms and tie rod
upper_lengths = [
    np.linalg.norm(upper_a_arm[1] - upper_a_arm[0]),  # Upper outer to inner leading
    np.linalg.norm(upper_a_arm[2] - upper_a_arm[0]),  # Upper outer to inner trailing
    np.linalg.norm(upper_a_arm[0] - lower_a_arm[0])   # Upper outer to lower outer
]

lower_lengths = [
    np.linalg.norm(lower_a_arm[1] - lower_a_arm[0]),  # Lower outer to inner leading
    np.linalg.norm(lower_a_arm[2] - lower_a_arm[0]),  # Lower outer to inner trailing
    np.linalg.norm(lower_a_arm[0] - tire_cop)         # Lower outer to tire center of pressure
]

tie_length = [
    np.linalg.norm(tie_rod[1] - tie_rod[0]),          # Tie rod outer to inner leading
    np.linalg.norm(upper_a_arm[0] - tie_rod[0]),      # Tie rod outer to upper outer
    np.linalg.norm(lower_a_arm[0] - tie_rod[0])       # Tie rod outer to lower outer
]

In [12]:
results = susprog.run_simulation (upper_a_arm, lower_a_arm, tie_rod, tire_cop, force_vector, susprog, roll_angles, jounce_displacements, lower_lengths, upper_lengths,tie_length)

In [13]:
# Extract the initial and final suspension points from the results
lower_initial = results.iloc[0, 2:11].values.reshape(3, 3)  # First row, lower A-arm (3x3)
upper_initial = results.iloc[0, 11:20].values.reshape(3, 3)  # First row, upper A-arm (3x3)

lower_final = results.iloc[-1, 2:11].values.reshape(3, 3)  # Last row, lower A-arm (3x3)
upper_final = results.iloc[-1, 11:20].values.reshape(3, 3)  # Last row, upper A-arm (3x3)

# Extract roll centers
rc_initial = results.iloc[0, 20:23].values  # Initial roll center
rc_final = results.iloc[-1, 20:23].values  # Final roll center

# Create a 3D scatter plot
fig = go.Figure()

# Add initial lower A-arm points
fig.add_trace(go.Scatter3d(
    x=lower_initial[:, 0], y=lower_initial[:, 1], z=lower_initial[:, 2],
    mode='markers+lines',
    marker=dict(size=5, color='blue'),
    name='Lower A-arm (Initial)'
))

# Add final lower A-arm points
fig.add_trace(go.Scatter3d(
    x=lower_final[:, 0], y=lower_final[:, 1], z=lower_final[:, 2],
    mode='markers+lines',
    marker=dict(size=5, color='red'),
    name='Lower A-arm (Final)'
))

# Add initial upper A-arm points
fig.add_trace(go.Scatter3d(
    x=upper_initial[:, 0], y=upper_initial[:, 1], z=upper_initial[:, 2],
    mode='markers+lines',
    marker=dict(size=5, color='green'),
    name='Upper A-arm (Initial)'
))

# Add final upper A-arm points
fig.add_trace(go.Scatter3d(
    x=upper_final[:, 0], y=upper_final[:, 1], z=upper_final[:, 2],
    mode='markers+lines',
    marker=dict(size=5, color='orange'),
    name='Upper A-arm (Final)'
))

# Add roll center points
fig.add_trace(go.Scatter3d(
    x=[rc_initial[0], rc_final[0]],
    y=[rc_initial[1], rc_final[1]],
    z=[rc_initial[2], rc_final[2]],
    mode='markers+lines',
    marker=dict(size=8, color='purple'),
    name='Roll Center'
))

# Configure the layout
fig.update_layout(
    scene=dict(
        xaxis_title='X-axis (m)',
        yaxis_title='Y-axis (m)',
        zaxis_title='Z-axis (m)'
    ),
    title="3D Suspension Geometry (Initial vs. Final)",
    legend=dict(x=0.1, y=0.9)
)

# Show the figure
fig.show()

"""
# Print the initial and final points
print("Initial Lower A-arm Points:\n", lower_initial)
print("Final Lower A-arm Points:\n", lower_final)

print("Initial Upper A-arm Points:\n", upper_initial)
print("Final Upper A-arm Points:\n", upper_final)

print("Initial Roll Center:", rc_initial)
print("Final Roll Center:", rc_final)
"""

'\n# Print the initial and final points\nprint("Initial Lower A-arm Points:\n", lower_initial)\nprint("Final Lower A-arm Points:\n", lower_final)\n\nprint("Initial Upper A-arm Points:\n", upper_initial)\nprint("Final Upper A-arm Points:\n", upper_final)\n\nprint("Initial Roll Center:", rc_initial)\nprint("Final Roll Center:", rc_final)\n'

In [14]:
# Plot RC vs roll

fig = go.Figure()

# Add the trace for "rc_z"
fig.add_trace(
    go.Scatter(
        x=results["roll_angle"],
        y=results["rc_z"],
        mode='lines',
        marker=dict(color='orange'),
        name='Roll Centre Location, Z'
    )
)

# Update layout to add a secondary y-axis
fig.update_layout(
    title='Roll Centre Location',
    xaxis=dict(title='Roll Angle (radians)'),
    yaxis=dict(
        title='RC Location, Z (m)',
        titlefont=dict(color='orange'),
        tickfont=dict(color='orange')
    ),
    legend=dict(x=0.5, y=-0.2, orientation='h')  # Adjust legend position if needed
)

# Show the figure
fig.show()

In [16]:
# Plot Bump steer vs roll

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=results["roll_angle"],
        y=results["Bump_Steer"],
        mode='lines',
        marker=dict(color='red'),
        name='Roll Centre Location, z',
        yaxis='y'  # Explicitly associate this trace with the primary y-axis
    )
)

# Update layout to add a secondary y-axis
fig.update_layout(
    title='bump',
    xaxis=dict(title='Jounce (m)'),
    yaxis=dict(
        title='RC Location, Z (m)',
        titlefont=dict(color='red'),
        tickfont=dict(color='red')
    ),
    legend=dict(x=0.5, y=-0.2, orientation='h')  # Adjust legend position if needed
)

# Show the figure
fig.show()


In [ ]:
# Plot inner positions vs roll

# Create the plot
fig = go.Figure()

# Add Z heights vs roll to the primary y-axis
fig.add_trace(
    go.Scatter(
        x=results["roll_angle"],
        y=results["lower_inner1_z"],
        mode='lines',
        marker=dict(color='red'),
        name='lower Leading Z Heights',
        yaxis='y'  # Primary y-axis
    )
)

fig.add_trace(
    go.Scatter(
        x=results["roll_angle"],
        y=results["upper_inner1_z"],
        mode='lines',
        marker=dict(color='orange'),
        name='Upper Leading Z Heights',
        yaxis='y'  # Primary y-axis
    )
)

# Add Y positions vs roll to the secondary y-axis
fig.add_trace(
    go.Scatter(
        x=results["roll_angle"],
        y=results["lower_inner1_y"],
        mode='lines',
        marker=dict(color='blue'),
        name='Lower Leading Y Positions',
        yaxis='y2'  # Secondary y-axis
    )
)
fig.add_trace(
    go.Scatter(
        x=results["roll_angle"],
        y=results["upper_inner1_y"],
        mode='lines',
        marker=dict(color='green'),
        name='Upper Leading Y Positions',
        yaxis='y2'  # Secondary y-axis
    )
)

# Update layout to add the secondary y-axis
fig.update_layout(
    title='Inner Leading Positions vs Roll',
    xaxis=dict(title='Roll Angles (degrees)'),
    yaxis=dict(
        title='Z Heights (m)',
        titlefont=dict(color='red'),
        tickfont=dict(color='red')
    ),
    yaxis2=dict(
        title='Y Positions (m)',
        titlefont=dict(color='blue'),
        tickfont=dict(color='blue'),
        anchor='x',
        overlaying='y',
        side='right'
    ),
    legend=dict(x=0.5, y=-0.2, orientation='h')  # Adjust legend position if needed
)

# Show the figure
fig.show()

In [ ]:
# Plot KPI/Caster vs roll

# Create the plot
fig = go.Figure()

# Add king pin incl
fig.add_trace(
    go.Scatter(
        x=results["roll_angle"],
        y=results["kingpin_inclination_deg"],
        mode='lines',
        marker=dict(color='red'),
        name='KPI (°)',
        yaxis='y'  # Primary y-axis
    )
)

# Add caster angle 
fig.add_trace(
    go.Scatter(
        x=results["roll_angle"],
        y=results["camber_angle_deg"],
        mode='lines',
        marker=dict(color='blue'),
        name='Caster Angle (°)',
        yaxis='y2'  # Secondary y-axis
    )
)

# Update layout to add the secondary y-axis
fig.update_layout(
    title='Inner Leading Positions vs Roll',
    xaxis=dict(title='Roll Angles (degrees)'),
    yaxis=dict(
        title='KPI [°]',
        titlefont=dict(color='red'),
        tickfont=dict(color='red')
    ),
    yaxis2=dict(
        title='Caster angle [°]',
        titlefont=dict(color='blue'),
        tickfont=dict(color='blue'),
        anchor='x',
        overlaying='y',
        side='right'
    ),
    legend=dict(x=0.5, y=-0.2, orientation='h')  # Adjust legend position if needed
)

# Show the figure
fig.show()

In [17]:
# Plot KPI/Caster vs jounce

# Create the plot
fig = go.Figure()

# Add king pin incl
fig.add_trace(
    go.Scatter(
        x=results["jounce"],
        y=results["kingpin_inclination_deg"],
        mode='lines',
        marker=dict(color='red'),
        name='lower Leading Z Heights',
        yaxis='y'  # Primary y-axis
    )
)

# Add caster angle 
fig.add_trace(
    go.Scatter(
        x=results["jounce"],
        y=results["camber_angle_deg"],
        mode='lines',
        marker=dict(color='blue'),
        name='Lower Leading Y Positions',
        yaxis='y2'  # Secondary y-axis
    )
)

# Update layout to add the secondary y-axis
fig.update_layout(
    title='Inner Leading Positions vs Roll',
    xaxis=dict(title='body jounce (m)'),
    yaxis=dict(
        title='KPI [°]',
        titlefont=dict(color='red'),
        tickfont=dict(color='red')
    ),
    yaxis2=dict(
        title='Caster angle [°]',
        titlefont=dict(color='blue'),
        tickfont=dict(color='blue'),
        anchor='x',
        overlaying='y',
        side='right'
    ),
    legend=dict(x=0.5, y=-0.2, orientation='h')  # Adjust legend position if needed
)

# Show the figure
fig.show()

In [18]:
# Plot Forces 

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create a 3x1 subplot with dual y-axes for each subplot
fig = make_subplots(
    rows=3, cols=1, 
    shared_xaxes=True,  # Share the x-axis
    subplot_titles=('', '', ''),  # Remove default subplot titles
    vertical_spacing=0.1,  # Adjust vertical spacing between subplots
    row_heights=[0.3, 0.3, 0.4],  # Adjust row heights to fit better
    specs=[
        [{'secondary_y': True}],  # Lower A-arm with dual y-axis
        [{'secondary_y': True}],  # Upper A-arm with dual y-axis
        [{'secondary_y': True}]   # Tie Rod with dual y-axis
    ]
)

# Lower A-arm Fx
fig.add_trace(
    go.Scatter(
        x=results["roll_angle"],
        y=results["Fx (Lower A-arm)"],
        mode='lines',
        marker=dict(color='red'),
        name='Fx (Lower A-arm)',
    ), row=1, col=1, secondary_y=False
)

# Lower A-arm Fy
fig.add_trace(
    go.Scatter(
        x=results["roll_angle"],
        y=results["Fy (Lower A-arm)"],
        mode='lines',
        marker=dict(color='blue'),
        name='Fy (Lower A-arm)',
    ), row=1, col=1, secondary_y=True
)

# Upper A-arm Fx
fig.add_trace(
    go.Scatter(
        x=results["roll_angle"],
        y=results["Fx (Upper A-arm)"],
        mode='lines',
        marker=dict(color='green'),
        name='Fx (Upper A-arm)',
    ), row=2, col=1, secondary_y=False
)

# Upper A-arm Fy
fig.add_trace(
    go.Scatter(
        x=results["roll_angle"],
        y=results["Fy (Upper A-arm)"],
        mode='lines',
        marker=dict(color='orange'),
        name='Fy (Upper A-arm)',
    ), row=2, col=1, secondary_y=True
)

# Tie Rod Fx
fig.add_trace(
    go.Scatter(
        x=results["roll_angle"],
        y=results["Fx (Tie Rod)"],
        mode='lines',
        marker=dict(color='purple'),
        name='Fx (Tie Rod)',
    ), row=3, col=1, secondary_y=False
)

# Tie Rod Fy
fig.add_trace(
    go.Scatter(
        x=results["roll_angle"],
        y=results["Fy (Tie Rod)"],
        mode='lines',
        marker=dict(color='brown'),
        name='Fy (Tie Rod)',
    ), row=3, col=1, secondary_y=True
)

# Update layout for different y-axes
fig.update_layout(
    title='Forces vs Roll Angles',
    height=900,  # Increased height for better readability
    showlegend=True,
    legend=dict(x=0.5, y=-0.1, orientation='h'),  # Adjust legend position
)

# Update y-axis titles
fig.update_yaxes(
    title_text="Fx (Lower A-arm)", 
    title_font=dict(color='red'), 
    tickfont=dict(color='red'), 
    row=1, col=1, secondary_y=False
)
fig.update_yaxes(
    title_text="Fy (Lower A-arm)", 
    title_font=dict(color='blue'), 
    tickfont=dict(color='blue'), 
    row=1, col=1, secondary_y=True
)
fig.update_yaxes(
    title_text="Fx (Upper A-arm)", 
    title_font=dict(color='green'), 
    tickfont=dict(color='green'), 
    row=2, col=1, secondary_y=False
)
fig.update_yaxes(
    title_text="Fy (Upper A-arm)", 
    title_font=dict(color='orange'), 
    tickfont=dict(color='orange'), 
    row=2, col=1, secondary_y=True
)
fig.update_yaxes(
    title_text="Fx (Tie Rod)", 
    title_font=dict(color='purple'), 
    tickfont=dict(color='purple'), 
    row=3, col=1, secondary_y=False
)
fig.update_yaxes(
    title_text="Fy (Tie Rod)", 
    title_font=dict(color='brown'), 
    tickfont=dict(color='brown'), 
    row=3, col=1, secondary_y=True
)

# Add small titles to each subplot
fig.add_annotation(
    text="Lower A-arm", 
    x=0, 
    y=1, 
    xref='paper', 
    yref='paper', 
    showarrow=False, 
    font=dict(size=10), 
    align='left', 
    xanchor='left', 
    yanchor='top',
    standoff=10,
    row=1, col=1
)
fig.add_annotation(
    text="Upper A-arm", 
    x=0, 
    y=1, 
    xref='paper', 
    yref='paper', 
    showarrow=False, 
    font=dict(size=10), 
    align='left', 
    xanchor='left', 
    yanchor='top',
    standoff=10,
    row=2, col=1
)
fig.add_annotation(
    text="Tie Rod", 
    x=0, 
    y=1, 
    xref='paper', 
    yref='paper', 
    showarrow=False, 
    font=dict(size=10), 
    align='left', 
    xanchor='left', 
    yanchor='top',
    standoff=10,
    row=3, col=1
)

# Update x-axis title
fig.update_xaxes(title_text="Roll Angles (degrees)", row=3, col=1)

# Show the figure
fig.show()